In [35]:
import re
from bs4 import BeautifulSoup

pattern_key_ability = r'<li><b>702\.\d+\.</b> <a href="/wiki/(\w+)"'
patter_reminder = re.compile(r'<i>\((.*?)\)</i>')
mtg_fandom_url = "https://mtg.fandom.com/wiki/"
missing = []

In [37]:
from bs4 import BeautifulSoup

def extract_types(s):
    types = re.findall(r'(\w+)\s*\(.*?\)', s)
    if len(types) == 0:
        return [s]
    return types

def ability_extraction(response_content, key_ability):
    ability = {}
    soup = BeautifulSoup(response_content, "html.parser")
    ability["meaning"] = "mtg:"+key_ability.lower()
    try:
        ability['description'] = soup.find('th', string='Reminder Text').find_next_sibling('td').text.replace("\n", "").replace("\u2019","").replace("\u2014","")
        ability['types'] = extract_types(soup.find('th', string='Type').find_next_sibling('td').text.replace("\n", ""))
        ability['written_as'], ability['description'] = ability['description'].split("(",1)
        ability['written_as'] = ability['written_as'].replace("\n", "").replace("  ", "")
    except:
        missing.append(key_ability)
        return ability
        
    if key_ability in ability['description'] and "only as a sorcery" not in ability['description'] or key_ability =="Equip":
        try:
            ability['description'], ability['alt_description'] = ability['description'].split(key_ability)
            ability['alt_written_as'] , ability['alt_description'] =  ability['alt_description'].split('(')
            ability['alt_written_as'] = key_ability + ability['alt_written_as']
            ability['alt_description'].replace(')', "")
        except:
            missing.append(key_ability)
            
    #rimuovi la ) finale
    ability['description'] = ability['description'][:-1]
    return ability

In [38]:
import requests, re, time
abilities = {}

response = requests.get(mtg_fandom_url+ "Keyword_ability")
if response.status_code == 200:
    key_ability_list = re.findall(pattern_key_ability, response.text)


for key_ability in key_ability_list:
    time.sleep(0.1)
    try:
        response = requests.get(mtg_fandom_url+key_ability)
    except Exception as e:
        print(e)
        print(key_ability)
    if response.status_code == 200:
        abilities[key_ability] = ability_extraction(response.content, key_ability)
        


In [23]:
# Aggiungo le keywords mancanti o errate
abilities["Miracle"] ={ #Miracle
        'written_as':"Miracle",
        'description': "You may reveal this card from your hand as you draw it if it’s the first card you’ve drawn this turn. When you reveal this card this way, you may cast it by paying [cost] rather than its mana cost",
        "types":['Static', 'Triggered']
    }
abilities["Visit"] ={ #Visit
        'written_as':"Visit",
        'description': "Whenever you roll to visit your Attractions, if the result is equal to a number that is lit up on this Attraction, [effect]",
        "types":['Triggered']
    }
abilities["Backup"] ={ #Backup
        'written_as':"Backup N",
        'description': 'When this creature enters the battlefield, put N +1/+1 counters on target creature. If that\’s another creature, it also gains the non-backup abilities of this creature printed below this one until end of turn.” Cards with backup have one or more abilities printed after the backup ability. (Some cards with backup also have abilities printed before the backup ability).',
        "types":['Triggered']
}
abilities["Craft"] = { #Craft
        'written_as':"Craft with [materials] [cost]",
        'description': "[Cost], Exile this permanent, Exile [materials] from among permanents you control and/or cards in your graveyard: Return this card to the battlefield transformed under its owner’s control. Activate only as a sorcery",
        "types":['Activated']
    }
abilities["Disguise"] = { #Disguise
        'written_as':"Disguise [cost]",
        'description': "You may cast this card as a 2/2 face-down creature with ward {2}, no name, no subtypes, and no mana cost by paying {3} rather than paying its mana cost",
        "types":['Static']
    }
abilities["Banding"] = { #Banding
        'written_as':"Banding [cost]",
        'description': "Any creatures with banding, and up to one without, can attack in a band. Bands are blocked as a group. If any creatures with banding you control are blocking or being blocked by a creature, you divide that creature's combat damage, not its controller, among any of the creatures it's being blocked by or is blocking",
        "types":['Static'],
        "alt_description" : '(Any [quality] creatures can attack in a band as long as at least one has "bands with other [quality]." Bands are blocked as a group. If at least two [quality] creatures you control, one of which has "bands with other [quality]," are blocking or being blocked by the same creature, you divide that creature\'s combat damage, not its controller, among any of the creatures it\'s being blocked by or is blocking.',
        "alt_written_as" : "Bands with other [quality]"
}


In [36]:
#Esporto in Yaml
import yaml
updatedKeywords = {"AbilityKeyword": {
    "see_also" : "mtg:Keyword_ability",
    'description' : "A passive ability represented by word that substitutes for a piece of rules text, such as deathtouch or flying.",
    'permissible_values' : abilities
}}
with open('keywords.yaml', 'w') as outfile:
    yaml.dump(updatedKeywords, outfile, default_flow_style=False)

In [ ]:
#Stampa anteprima:
print(yaml.dump(updatedKeywords))